In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Layer
import keras
from keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler

from tensorflow.keras.layers.experimental import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer as Vectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import warnings
warnings.filterwarnings('ignore')


In [2]:
# First, we define the attention class. It was implemented from the Bahdanau et al 2014 research paper on attention mechanism.

class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

   

  def call(self, enc_hidden, dec_inter_hidden):
    # hidden here means 'hidden_size', word length the same as 'sentence length'
    # enc_hidden shape == (batch_size, word_length, hidden)
    # dec_inter_hidden shape == (batch_size, word_length, hidden)
    
    # attention_hidden_layer shape == (batch_size, word_length, units)
    #dec_inter_hidden = tf.expand_dims(dec_inter_hidden, 1)
    attention_hidden_layer = (tf.nn.tanh(self.W1(enc_hidden) +
                                         self.W2(dec_inter_hidden)))

    # score shape == (batch_size, word_length, 1)
    # Score is a function of the intermediate hidden of the decoder and the encoder hidden
    # This gives you an unnormalized score for each word.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, word_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden)
    context_vector = attention_weights * enc_hidden # (batch_size, word_length, hidden)
    context_vector = tf.reduce_sum(context_vector, axis=1) #(batch_size, hidden)

    return context_vector, attention_weights

In [3]:
# Build encoder model

class Encoder(tf.keras.Model):
  def __init__(self, units,embedding_dim,src_vocab_size, mt_vocab_size,dropout=0.2):
    super(Encoder, self).__init__()
    # Define bidirectional encoder for source language
    self.src_encoder = layers.Bidirectional(layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',
                                   dropout= dropout))
    
    # Define bidirectional encoder for machine translated output language
    self.mt_encoder = layers.Bidirectional(layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',
                                   dropout = dropout))
    # Embedding layer for both src and mt
    self.src_embed_layer = tf.keras.layers.Embedding(src_vocab_size, embedding_dim,mask_zero=True)
    self.mt_embed_layer = tf.keras.layers.Embedding(mt_vocab_size, embedding_dim, mask_zero=True)

    # Embed dropout
    self.src_embed_dropout = tf.keras.layers.Dropout(dropout)
    self.mt_embed_dropout = tf.keras.layers.Dropout(dropout)

    # Define dropout 
    self.dropout = tf.keras.layers.Dropout(dropout)
    #self.embed_dropout  = tf.keras.layers.Dropout(dropout)

  def call(self, src, mt):
    # Embed both src and mt words:
    src = self.src_embed_layer(src)
    mt = self.mt_embed_layer(mt)

    # Add dropout
    src = self.src_embed_dropout(src)
    mt = self.mt_embed_dropout(mt)

    # first, we encode both src and mt:
    src, src_state,_ = self.src_encoder(src)
    mt, mt_state,_ = self.mt_encoder(mt)

    # Apply shared dropout to both src_encode and mt_encode
    src = self.dropout(src)
    mt = self.dropout(mt)

    return src , mt, mt_state

In [7]:
# Build decoder model
class Decoder(tf.keras.Model):
  def __init__(self,units,att_units, linear_units, vocab_size,embedding_dim,dropout =0.2):
    # Initialize with parent class
    super(Decoder, self).__init__()
    self.units = units
    # embedding layer for target words
    self.embed  = tf.keras.layers.Embedding(vocab_size, embedding_dim,mask_zero=True)

    # Decoder uses two GRUs to compute the probability of the next post-edited word
    self.gru1 = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.gru2 = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',
                                    dropout = dropout)
    
    #Define attention class for source sentence embedding:
    self.src_attention = BahdanauAttention(att_units)
    # Define attention class for machine translated sentence embedding:
    self.mt_attention = BahdanauAttention(att_units)

    # Define linear transformation for final concatenated context.
    self.linear = tf.keras.layers.Dense(linear_units)    
    
    # Define embed dropout layer
    self.embed_dropout = tf.keras.layers.Dropout(dropout)
    # Define context dropout layer
    self.dropout = tf.keras.layers.Dropout(dropout)

    # Define fully connected layer
    self.fc = tf.keras.layers.Dense(vocab_size)

  
    
  def call(self, prev_word, hidden_state, src_encode ,mt_encode ):
    # This decoder model predicts the next word in 2 steps:
    # First, it uses the previous word embedding (prev_word_embed) and hidden state (hidden_state) to compute an intermediate state.
    # second, it uses the new intermediate hidden state (inter_hidden)and the context_vector to predict the next word in the target sentence (hpe)

    # Get embedding of previous word , shape = (batch_size, 1, embedding_dim)
    prev_word = self.embed(prev_word)

    # Add dropout to embed layer
    prev_word = self.embed_dropout(prev_word)

    # Expand the hidden state by 1 more dimension (batch_size, 1, hidden)
    hidden_state = tf.expand_dims(hidden_state,1)

    # we first merge the previous word embedding and hidden_state of the second gru
    #hidden_merge shape = (batch_size, 1, hidden + embedding_dim)
    hidden_merge  = tf.concat([tf.expand_dims(prev_word,1), hidden_state], -1)

    # pass to the gru, inter_hidden shape = (batch_size, units)
    #hidden_merge = tf.expand_dims(hidden_merge,1)
    inter_hidden , state = self.gru1(hidden_merge) 
    #inter_hidden  = tf.squeeze(inter_hidden)
    
    # Use attention mechanism to extract context from each encoded entities:
    src_context, _  = self.src_attention(src_encode, inter_hidden)
    mt_context, _  = self.mt_attention(mt_encode, inter_hidden)

    #print(src_context.shape, mt_context.shape)
    # Merge both context and transform result linearly:
    # merged_context = tf.concat([src_context, mt_context],-1)
    context_vector = self.linear( tf.concat([src_context, mt_context],-1))

    # Apply dropout to the context:
    context_vector = self.dropout(context_vector)

    # concatenate intermediate state and context_vector then, pass to gru2
    # Expand context by# Define the loss class

    context_vector = tf.expand_dims(context_vector, 1)

    #hidden_context_merge = tf.concat([inter_hidden, context_vector], -1)
    #hidden_context_merge = tf.expand_dims(hidden_context_merge,1)
    final_hidden, state = self.gru2(tf.concat([inter_hidden, context_vector], -1))
    #print(final_hidden.shape, inter_hidden.shape)
    # Run through a fully connected layer to predict logits (before softmax) of next word:
    # output shape = (batch_size, vocab_size)
    output = self.fc(final_hidden)

    # return output and hidden state of gru2
    return output, state


In [9]:

class MaskedLoss(tf.keras.losses.Loss):
  def __init__(self):
    self.name = 'masked_loss'

    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')

  def __call__(self, y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss = self.loss(y_true, y_pred)

    # Mask off the losses on the elements that were padded in the hpe sentences.
    mask = tf.cast(y_true != 0, tf.float32)
    loss *= mask

    # Return the mean loss.
    return tf.reduce_mean(loss) #one more dimension first

# Define full model
class APE(tf.keras.Model):
  def __init__(self,units,embedding_dim, att_units, linear_units,
               src_text_processor, mt_hpe_text_processor, dropout=0.2):
    # initialize with parent class:
    super(APE, self).__init__()

    # initialize encoder:
    self.encoder  = Encoder(units,embedding_dim,src_text_processor.vocabulary_size(),
                            mt_hpe_text_processor.vocabulary_size(),dropout= dropout)

    # Initialize decoder:
    self.decoder  = Decoder(units, att_units, linear_units, mt_hpe_text_processor.vocabulary_size(), 
                            embedding_dim,dropout =dropout)

    # Set the preprocessors for both source , machine translated and human post edited sentences:s
    self.src_processor = src_text_processor
    self.mt_hpe_processor = mt_hpe_text_processor

  def _preprocess(self,src , mt , hpe):
    # This function preprocesses the source, mt and hpe sentences into respective tokens using the preprocessors.
    src_token = self.src_processor(src)
    mt_token = self.mt_hpe_processor(mt)
    hpe_token = self.mt_hpe_processor(hpe)
    return src_token, mt_token , hpe_token

  # Define the customized training function:
  @tf.function
  def train_step(self, data):
    # Unpack the data.
    src , mt, hpe = data

    # Preprocess source, mt and hpe sentences:
    src , mt , hpe = self._preprocess(src, mt , hpe)
    
    max_target_length = tf.shape(hpe)[1]
   
    # Set loss to 0.0
    loss = tf.constant(0.0)

    # Set the first input to the decoder model to the '[START]' token which is the first element of each sentence:
    dec_input = hpe[:,0]

    with tf.GradientTape() as tape:       
      # Pass the source and machine translated sentences to the encoder as inputs.
      # we use the returned hidden state of the encoder to initialize the decoder's hidden state:
      src_encode , mt_encode , dec_hidden = self.encoder(src, mt)

      # Use for loop to iterate through each word as we predict the next word:
      for i in range(1,max_target_length):
      # Make predictions
        # Pass dec_input (serves as previous words), dec_hidden and the encoded src and mt to the decoder.
        predictions , dec_hidden = self.decoder(dec_input, dec_hidden, src_encode, mt_encode)

        # Calculate the loss
        loss  = loss + self.loss(hpe[:,i],predictions)

        # Using teacher forcing:
        # Teacher forcing continually feeds the next correct word in the real hpe (target) sentences to the model
        # instead of passing what the model predicted back into the model.
        dec_input = hpe[:, i]

        
      
      # Average the loss over all non padding tokens.
      average_loss = loss / tf.cast(max_target_length, tf.float32)

    # Apply an optimization step
    variables = self.trainable_variables 
    gradients = tape.gradient(average_loss, variables)
    self.optimizer.apply_gradients(zip(gradients, variables))

    # Return a dict mapping metric names to current value
    return {'batch_loss': average_loss}
    

    

In [12]:
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)



class APETranslator(tf.Module):
  def __init__(self,
               encoder, decoder, 
               src_text_processor,
               mt_hpe_text_processor):
    # Initialize encoder, decoder, src_processor, mt_hpe_processor:
    self.encoder = encoder
    self.decoder = decoder
    self.src_text_processor = src_text_processor
    self.mt_hpe_text_processor = mt_hpe_text_processor

    # We initialize a string look up that will convert tokens to their equivalent words from the src and mt vocabulary:
    # We use the StringLookup class.
    self.output_token_string_from_index = (
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary= mt_hpe_text_processor.get_vocabulary(),
            invert=True))

    # The output should never generate padding, unknown, or start.
    # Therefore, get the index of '', '[UNK]' and '[START]' from the vocabulary:
    index_from_string = tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary= mt_hpe_text_processor.get_vocabulary())
    token_mask_ids = index_from_string(['',
                                        '[UNK]',
                                        '[START]']).numpy()

    # Create a token mask by first creating an array of tokens (vocabulary) and then initialize all to False
    # Set the indexes of the '', '[UNK]' and '[START]' to True:
    token_mask = np.zeros([index_from_string.vocabulary_size()], dtype=np.bool)
    token_mask[np.array(token_mask_ids)] = True

    # set as a property of the class
    self.token_mask = token_mask

    # Get the index of the '[START]' and '[END]' tokens:
    
    self.start_token = index_from_string('[START]')
    self.end_token = index_from_string('[END]')

  def tokens_to_text(self, result_tokens):
    # This method converts index tokens to the string tokens by using the stringlookup we initialized in the init() method:
    result_text_tokens = self.output_token_string_from_index(result_tokens)
    
    # Join tokens to form a full string separating with a ' ':
    result_text = tf.strings.reduce_join(result_text_tokens,
                                        axis=1, separator=' ')

    # Strip any extra ' ' from the text:
    result_text = tf.strings.strip(result_text)
   
    return result_text

  def sample(self, logits, temperature):

    # Add 2 new axis to the token_mask shape
    token_mask = self.token_mask[tf.newaxis, tf.newaxis, :]

    # Set the logits for all masked tokens ('','[START]', '[UNK]') to -inf, so they are never chosen.
    # The logits (shape== (batch_size, vocab_size) here represents the independent probabilities of each word in the vocabulary.
    logits = tf.where(token_mask, -np.inf, logits)
    # If temperature is 0, get the max logit argument for each sample in the batch:
    # This argument represents the index of the word with the highest independent probability.
    if temperature == 0.0:
      new_tokens = tf.argmax(logits, axis=-1)
    # Else, sample from the independent probabilities of the logit:
    else: 
      logits = tf.squeeze(logits, axis=1)
      new_tokens = tf.random.categorical(logits/temperature,
                                          num_samples=1)
    return new_tokens

  def single_post_edit(self, src, mt, max_length=50, temperature=1.0):
    result = []
    src = self.src_text_processor(src)
    mt = self.mt_hpe_text_processor(mt)
    src, mt, dec_hidden= self.encoder(src, mt)
    dec_input = tf.fill((1,),self.start_token)
    #tf.expand_dims([tokenizer.word_index['<start>']], 0)
    
    for i in range(max_length):
        # TensorShape([1, 1, 15000]), TensorShape([1, 1024]))
        predictions, dec_hidden = self.decoder(dec_input, dec_hidden, src, mt)

        predicted_id = tf.random.categorical(tf.squeeze(predictions,1)/temperature, 1)[0][0].numpy()
        #print(predicted_id)
        
        result.append(self.output_token_string_from_index(tf.cast(predicted_id, tf.int64)).numpy())

        if predicted_id == self.end_token:
            return b' '.join(result[:-1])

        dec_input = tf.fill((1,), predicted_id)
    
    return b' '.join(result)

  #@tf.function
  def post_edit(self, src, mt, max_length=50,temperature=1.0):

    batch_size = tf.shape(src)[0]

    # Encode the input
    src_tokens = self.src_text_processor(src)
    mt_tokens = self.mt_hpe_text_processor(mt)

    # Encode inputs:
    src_enc, mt_enc , dec_state = self.encoder(src_tokens, mt_tokens)

    # Initialize the decoder
    new_tokens = tf.fill((batch_size,), self.start_token)


    # Initialize the accumulators for the decoder output:
    result_tokens = tf.TensorArray(tf.int64, size=1, dynamic_size=True)
    # Initialize a tf array that tracks the end_token of all samples in the batch:
    done = tf.zeros([batch_size, 1], dtype=tf.bool)
    #print(f'outside loop: {new_tokens.shape}', dec_state.shape)

    for t in tf.range(max_length):
      # Pass through decoder model:
      #print('yes')
      #print('Beginning of loop',new_tokens.shape)
      dec_result, dec_state = self.decoder(new_tokens, dec_state, src_enc, mt_enc)
      #print('NO')
      new_tokens = self.sample(dec_result, temperature)
      #print('After self.sample',new_tokens.shape)
      # If a sequence produces an `end_token`, set it `done`
      done = done | (new_tokens == self.end_token)
      # Once a sequence is done it only produces 0-padding.
      new_tokens = tf.where(done, tf.constant(0, dtype=tf.int64), new_tokens)

      # Collect the generated tokens
      result_tokens = result_tokens.write(t, new_tokens)
      new_tokens = tf.squeeze(new_tokens)

      if tf.reduce_all(done):
        break

    # Convert the list of generates token ids to a list of strings.
    result_tokens = result_tokens.stack()
    result_tokens = tf.squeeze(result_tokens, -1)
    result_tokens = tf.transpose(result_tokens, [1, 0])

    result_text = self.tokens_to_text(result_tokens)

    return {'text': result_text}

# Define a function to always get an APE Translator object from a trained model ( useful in the online learning section):
def get_post_editor(model):
  return APETranslator(ape.encoder, ape.decoder, ape.src_processor, ape.mt_hpe_processor)



def get_bleu_score(y_true, y_pred):
  # This function calculates the cummulative BLEU-4 score by default:
  #y_pred = y_pred.numpy()
  scores = []
  #smooth_func = SmoothingFunction()
  # Get human post edit and machine post edit from both y_true, y_pred
  for hpe, mpe in zip(y_true, y_pred):
    hpe = [hpe.split(b' ')]
    mpe = mpe.split(b' ')
    score = sentence_bleu(hpe,mpe)
    scores.append(score)
  return np.array(scores).mean()

In [13]:

def TER_score(y_true, y_pred):
  # This function calculates the cummulative TER score by default:
  #y_pred = y_pred.numpy()
  scores = []
  # Get human post edit and machine post edit from both y_true, y_pred
  for hpe, mpe in zip(y_true, y_pred):
    score = pyter.ter(mpe.split(),hpe.split())
    scores.append(score)
  return np.array(scores).mean()

def get_blue_ter(y_true,y_pred):
  bleu_score = get_bleu_score(y_true, y_pred)
  ter_score = TER_score(y_true, y_pred)

  print(f'BLEU Score: {bleu_score:.2f}')
  print(f'TER Score:  {ter_score:.2f}')

In [14]:
# Function removers the new line character from each sentence in the dataset:
def preprocess(text):
  text = [t.replace(b'\n',b'') for t in text]
  text = [t.lower() for t in text]
  return text


# Define a sort_score function to sort the datapoints.
def sort_score(scores):
  result = []
  for index, score in enumerate(list(scores[0])):
    result.append((index,score))
  result.sort(key=lambda x: x[1],reverse=True)
  return result



In [15]:
# Initialize the learning rate for the model.
initial_learning_rate = 0.001
# We schedule a learning rate for every epoch using the formula below:
# For every epoch, learning rate is reduced using this formula.


# We use a time based decay to schedule the learning rate which will be appropriate for online learning.
def lr_time_based_decay2(epoch, lr):
    return lr * 1 / (1 +  epoch)


def set_hyperparameters(score, threshold,initial_learning_rate):
  # Get all values above threshold:
  score = score[score[:,1] >= threshold]
  # Get mean score:
  score = score[:,1]
  mean = score.mean()
  if mean >= 0.85:
    epochs= 8
  elif mean >= 0.65:
    epochs = 5
  else:
    epochs = 3
  return epochs 


def model_compile(model,learning_rate):
  model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
    loss=MaskedLoss(),
    )
  return model

def get_human_post_edit(src,ape):
  print(f'Source sentence: {src}')
  print(f'Machine post-edit translation: {ape}')
  print('')
  satisfied = input('Are you satisfied with the translation?: (Y for Yes and N for No) \t')

  if satisfied == 'Y':
    #print('yes')
    train = False
    #ape = ape.encode('utf-8')
    return [ape], train
  elif satisfied == 'N':
    #print('No')
    good_answer  =  input('Please provide yours below:\n')
    while (good_answer == '' or good_answer == ' '):
      good_answer = input('Please provide a valid translation below:')
    good_answer  = good_answer.encode('utf-8')
    train = True
    return [good_answer], train
  else:
    #print('nothing')
    train = False
    return [''], train
  
 
def get_datastream(src, mt, hpe, batch_size=5):
  buffer_size = len(src)
  BATCH_SIZE = batch_size

  # This Dataset class shuffles the data and precreates batches for it.
  dataset = tf.data.Dataset.from_tensor_slices((src, mt, hpe)).shuffle(buffer_size)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
  return dataset

def online_ape( sent_stream, ape_model, vectorizer,ckpt_manager=None, 
               data=None, epochs= 3,threshold=0.35,initial_learning_rate=0.001, interactive=False):
  '''
    This function is implemented from Matteo Negri et al (2017) as explained above.
    Args:
    sent_stream: development dataset for online training (i.e src, mt ,hpe)
    ape_model: new or pretrained ape model. If new model, data cannot be empty.
    ckpt_manager: checkpoint manager.
    vectorizer: vectorizer object that vectorizes sentences.
    data: knowledge base (kb) that stores all the processed/learned sent_stream sets and/or ape training data in format (src, mt , hpe).
    epochs: Number of iterations to train the model on data.
    threshold: similarity score threshold to use when retrieving similar datapoints.
    interactive: Used to interact directly with any user who chooses to provide the right human post edit for every src, mt pair.
  '''
  # Unpack
  if not interactive:
    src, mt, hpe = sent_stream
    # Reverse list for popping:
    src.reverse()
    mt.reverse()
    hpe.reverse()
  else:
    src, mt = sent_stream
    # Reverse list for popping:
    src.reverse()
    mt.reverse()

  # while sentence still remains in the stream, pop one sentence from the stream:
  while(len(src) != 0):
    # Get one sentence at a time:
    src_sent = [src.pop()] 
    mt_sent = [mt.pop()] 
    # Retrieve sentences similar to this sentence from the dataset:
    # If no dataset (knowldedge base):
    if not data:
      # Make copy of generic model and predict translation:
      #model = deepcopy(ape_model)
      # Create the translator version of this generic model:
      editor = get_post_editor(ape_model)
      
    else:
      # Vectorize source sentence:
      sentence = vectorizer.transform(src_sent)
      # Calculate the similarity between sentence and dataset source sentences:
      similarity_score = cosine_similarity(sentence, vectorizer.transform(np.array(data[0])))
      # sort similarity scores with respective index and convert to an np array:
      similarity_score = np.array(sort_score(similarity_score))
      # Get relevant index of sentences above threshold value:
      relevant_index = similarity_score[similarity_score[:,1] >= threshold]

      # Get the relevant samples from the data using the index:
      relevant_src = np.array(data[0])[relevant_index[:,0].astype('int32')]
      relevant_mt = np.array(data[1])[relevant_index[:,0].astype('int32')]
      # ind = relevant_index[:,0].astype('int32')
      # print('Indexes', ind)
      # for i in range(len(ind)):
      #   print(np.array(data[2])[i])
      #print(np.array(data[2])[relevant_index[:,0].astype('int32')])
      relevant_hpe = np.array(data[2])[relevant_index[:,0].astype('int32')]

      #relevant_src[0].split())

      if relevant_src.shape[0] >= 3:
        #print('Sample size: ', relevant_src.shape[0])
        # Dynamically get number of epochs and learning rate. Then compile model using the learning_rate:
        epochs = set_hyperparameters(similarity_score, threshold, initial_learning_rate)
        #ape_model = model_compile(ape_model,learning_rate)
        
        # Locally adapt similar sentences to model:
        print(f'Training on {relevant_src.shape[0]} similar sample sentences...')
        # Get dataset object:
        dataset = get_datastream(list(relevant_src), list(relevant_mt), list(relevant_hpe))
        # Train model:
        ape_model.fit(dataset,epochs=epochs,
                      callbacks=[LearningRateScheduler(lr_time_based_decay)])

      # Get translator version of the trained model:
      editor  = get_post_editor(ape_model)

    # If dataset is empty, create the right structure for it:
    if not data:
      data = [[],[],[]]

    if interactive :
      # Predict translation of the sentence:
      pe = editor.single_post_edit(src_sent,mt_sent)
      #pe = pe.encode('utf-8')
      #predictions.append(pe)
      hpe_sent , train = get_human_post_edit(src_sent[0], pe)
    else:
      train = True
      hpe_sent = [hpe.pop()]

    # Update model according to new sentence if train is set to True
    if train:
      # Get dataset object:
      dataset = get_datastream(src_sent,mt_sent,hpe_sent, batch_size=1)
      # Train model:
      ape_model.fit(dataset, epochs=epochs, callbacks=[LearningRateScheduler(lr_time_based_decay2)])
    # Append the result to the dataset knowledge base (data):
    data[0].extend(src_sent)
    data[1].extend(mt_sent)
    if hpe_sent[0] == '':
      data[2].extend([pe])
    else:
      data[2].extend(hpe_sent)
    

  if ckpt_manager:
    print('Online training completed. Saving model parameters...')

    ckpt_manager.save()

  return 




  
  
